# Models
(arranjar um nome melhor para o notebook)

Notebook para o treinamento de modelos

In [1]:
import random

# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# models
import sklearn
import sktime
import keras
import statsmodels.api as sm
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from tensorflow import keras
from tensorflow.keras import layers

# model selection and metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import MeanAbsoluteError

# constants
PROCESSED_DATA_PATH = '../data/processed-data/'
TARGET_COLS = ['target1', 'target2', 'target3', 'target4']
TEST_SPLIT_DATE = '2021-04-30'

# disable warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.options.mode.chained_assignment = None  # default='warn'

2023-03-26 15:05:18.707363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 15:05:18.865451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-26 15:05:18.865469: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-26 15:05:19.550312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Auxiliary Functions

In [2]:
def plot_time_series(df, groupby = 'median', cols = TARGET_COLS):
    df_melted = df.groupby('Dt').median()[cols]\
    .melt(var_name='target', value_name='value', ignore_index=False)
    df_melted.reset_index(inplace=True)

    sns.set(rc={'figure.figsize':(10,20)})
    sns.FacetGrid(df_melted, col='target', col_wrap=2, height=9,aspect=2, sharey=False)\
    .map(sns.lineplot, 'Dt', 'value');

In [3]:
def plot_train_test_pred(train, test, pred = pd.DataFrame(), groupby = 'median', cols = TARGET_COLS):
    train['type'] = 'train'
    test['type'] = 'test'
    pred['type'] = 'pred'
    
    cols.append('type')

    df = pd.concat([train, test, pred], axis=0)
    df_melted = df.groupby('Dt').median()[cols]\
        .melt(var_name='target', value_name='value', ignore_index=False)
    df_melted.reset_index(inplace=True)
    
    sns.set(rc={'figure.figsize':(10,20)})
    sns.FacetGrid(df_melted, col='target', col_wrap=2, height=9,aspect=2, sharey=False)\
        .map(sns.lineplot, 'Dt', 'value', hue='type');

## Data Preparation

Apenas realizar modificações necessárias para os modelos como *splitting* dos datasets e normalização. Toda a limpeza de dados deve ser feita em **/src/data-engineering/data-preparation**

### Loading the data

In [4]:
%%time
df_targets = pd.read_pickle(PROCESSED_DATA_PATH + 'targets.pkl')

CPU times: user 188 ms, sys: 134 ms, total: 322 ms
Wall time: 321 ms


In [5]:
df_targets

,Dt,IdPlayer,target1,target2,target3,target4,IdDtPlayer
0,2018-01-01,628317,0.011167,4.474708,0.005168,5.735294,2018-01-01 00:00:00_628317
1,2018-01-01,547989,0.042993,5.593385,0.045033,2.794118,2018-01-01 00:00:00_547989
2,2018-01-01,519317,0.974327,56.177044,13.693746,64.166664,2018-01-01 00:00:00_519317
3,2018-01-01,607625,0.006700,2.675097,0.005168,1.862745,2018-01-01 00:00:00_607625
4,2018-01-01,592547,0.001117,0.632296,0.002953,0.931373,2018-01-01 00:00:00_592547
...,...,...,...,...,...,...,...
2695783,2021-07-31,527055,0.000000,0.027442,0.000000,0.157926,2021-07-31 00:00:00_527055
2695784,2021-07-31,542963,0.000834,0.137212,0.000000,0.146908,2021-07-31 00:00:00_542963
2695785,2021-07-31,622259,0.000119,0.102909,0.000000,0.058763,2021-07-31 00:00:00_622259
2695786,2021-07-31,642840,0.000119,0.048024,0.000000,0.036727,2021-07-31 00:00:00_642840


### Sorting by IdPlayer

In [6]:
# sort by player then by date
df_targets.sort_values(by=['IdPlayer', 'Dt'], inplace=True)
# reset index
df_targets.reset_index(drop=True, inplace=True)
df_targets

In [8]:
# Example dataset
# df_targets = df_targets[df_targets['IdPlayer'].isin([112526, 685503, 628317])]
# df_targets

### Shifting

In [10]:
%%time
# Make the shifts for each target
shift_vals = [1, 2, 3, 4, 5, 6, 7, 14, 30]
df = pd.DataFrame()
# Iterate over players to make the shift only using the player data
for player in df_targets['IdPlayer'].unique():
    df_player = df_targets[df_targets['IdPlayer'] == player]
    # Iterate over the pre-defined shift values
    for shift_val in shift_vals:
        # Iterate over the targets
        for target in TARGET_COLS:
            # Make the shift
            df_player[f'{target}_shift_{shift_val}'] = df_player[target].shift(shift_val)
    # Concatenate the player data with the rest of the data
    df = pd.concat([df, df_player], axis=0)
    # Remove the player data from memory
    del df_player
# Remove the original targets from memory
del df_targets

CPU times: user 4min 39s, sys: 1.68 s, total: 4min 41s
Wall time: 4min 41s


In [13]:
df[df['IdPlayer'] == 628317].head(10)

,Dt,IdPlayer,target1,target2,target3,target4,IdDtPlayer,target1_shift_1,target2_shift_1,target3_shift_1,...,target3_shift_7,target4_shift_7,target1_shift_14,target2_shift_14,target3_shift_14,target4_shift_14,target1_shift_30,target2_shift_30,target3_shift_30,target4_shift_30
1806348,2018-01-01,628317,0.011167,4.474708,0.005168,5.735294,2018-01-01 00:00:00_628317,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806349,2018-01-02,628317,0.006356,2.521008,0.001648,2.766917,2018-01-02 00:00:00_628317,0.011167,4.474708,0.005168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806350,2018-01-03,628317,0.003556,2.380952,0.003624,3.439381,2018-01-03 00:00:00_628317,0.006356,2.521008,0.001648,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806351,2018-01-04,628317,0.003363,3.332367,0.012294,4.849601,2018-01-04 00:00:00_628317,0.003556,2.380952,0.003624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806352,2018-01-05,628317,0.023976,1.440761,0.001501,2.371288,2018-01-05 00:00:00_628317,0.003363,3.332367,0.012294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806353,2018-01-06,628317,0.032154,2.808843,0.004137,3.378956,2018-01-06 00:00:00_628317,0.023976,1.440761,0.001501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806354,2018-01-07,628317,0.009796,5.736138,0.118516,6.114551,2018-01-07 00:00:00_628317,0.032154,2.808843,0.004137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806355,2018-01-08,628317,0.007320,11.475410,43.574837,4.285714,2018-01-08 00:00:00_628317,0.009796,5.736138,0.118516,...,0.005168,5.735294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806356,2018-01-09,628317,0.005797,3.615257,9.716749,1.200418,2018-01-09 00:00:00_628317,0.007320,11.475410,43.574837,...,0.001648,2.766917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806357,2018-01-10,628317,0.022303,2.240281,1.752281,1.845444,2018-01-10 00:00:00_628317,0.005797,3.615257,9.716749,...,0.003624,3.439381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df[df['IdPlayer'] == 112526].head(5)

,Dt,IdPlayer,target1,target2,target3,target4,IdDtPlayer,target1_shift_1,target2_shift_1,target3_shift_1,...,target3_shift_7,target4_shift_7,target1_shift_14,target2_shift_14,target3_shift_14,target4_shift_14,target1_shift_30,target2_shift_30,target3_shift_30,target4_shift_30
0,2018-01-01,112526,0.055277,5.496109,0.025839,16.176470,2018-01-01 00:00:00_112526,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,112526,0.060625,3.252914,0.030486,8.541353,2018-01-02 00:00:00_112526,0.055277,5.496109,0.025839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-03,112526,0.029341,1.648352,0.032613,10.490111,2018-01-03 00:00:00_112526,0.060625,3.252914,0.030486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-04,112526,0.014799,2.665894,0.087422,19.091467,2018-01-04 00:00:00_112526,0.029341,1.648352,0.032613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-05,112526,0.083916,1.161002,0.024759,6.643879,2018-01-05 00:00:00_112526,0.014799,2.665894,0.087422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
pd.to_pickle(df, PROCESSED_DATA_PATH + 'shifted_targets.pkl')

### Train Test Split

In [17]:
train = df[df['Dt'] <= '2021-04-30']
y_train = train[TARGET_COLS]
x_train = train.drop(TARGET_COLS, axis=1)
test = df[df['Dt'] > TEST_SPLIT_DATE]
y_test = test[TARGET_COLS]
x_test = test.drop(TARGET_COLS, axis=1)
del df

NameError: name 'df' is not defined

In [20]:
train.shape

(2506176, 43)

In [21]:
test.shape

(189612, 43)

### Data Normalization

In [ ]:
!

## Training

In [ ]:
# def train_model(model, train, test, y_cols = ['target1', 'target2', 'target3', 'target4']):
#     results = {}
#     for col in y_cols:
#         y_train = train[col].set_axis(train['Dt'])
#         y_test = test[col].set_axis(test['Dt'])
#         model.fit(y_train)
#         pred = model.predict(y_test)
#         results[col] = pred
#     return results

In [ ]:
# def train_baseline_model(model, train, fh_size, y_cols = ['target1', 'target2', 'target3', 'target4']):
#     results = {}
#     fh = np.arange(len(fh_size)) + 1
#     for col in y_cols:
#         y_train = train[col].set_axis(train['Dt'])
#         model.fit(y_train)
#         pred = model.predict(fh)
#         results[col] = pred
#     return results

In [ ]:
# # Naive
# naive = NaiveForecaster(strategy='last').fit(train)
# y_pred_naive = naive.predict(test)

# # Mean
# mean = NaiveForecaster(strategy="mean").fit(train)
# y_pred_mean = mean.predict(test)

# # Drift
# drift = NaiveForecaster(strategy="drift").fit(train)
# y_pred_drift = drift.predict(test)

### Baseline Models

In [24]:
def naive(test):
    y_pred = pd.DataFrame(columns=TARGET_COLS)
    for target in TARGET_COLS:
        y_pred[target] = test[target + '_shift_1']
    return y_pred

In [25]:
y_pred = naive(test)
y_pred

,target1,target2,target3,target4
1216,0.009073,0.412742,0.020685,100.000000
1217,0.006267,0.643099,0.071587,100.000000
1218,0.011920,0.605747,0.036597,100.000000
1219,0.003875,0.488195,0.072502,44.480946
1220,0.004221,0.163703,0.018476,11.722289
...,...,...,...,...
2695783,0.000661,0.109511,0.000882,0.545347
2695784,0.000642,0.182785,0.000482,0.574713
2695785,0.000515,0.144995,0.000681,0.068164
2695786,0.000536,0.139173,0.000389,0.079939


In [27]:
test[TARGET_COLS]

,target1,target2,target3,target4
1216,0.006267,0.643099,0.071587,100.000000
1217,0.011920,0.605747,0.036597,100.000000
1218,0.003875,0.488195,0.072502,44.480946
1219,0.004221,0.163703,0.018476,11.722289
1220,0.007261,0.300970,0.015403,3.065583
...,...,...,...,...
2695783,0.000642,0.182785,0.000482,0.574713
2695784,0.000515,0.144995,0.000681,0.068164
2695785,0.000536,0.139173,0.000389,0.079939
2695786,0.000452,0.193631,0.002074,0.217098


### testing the functions

### .

## Model Evaluation